In [1]:
!pip install beautifulsoup4 openpyxl requests pandas


### 다음뉴스 크롤링 코드

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Base URL
base_url = "https://news.daum.net/breakingnews/digital?page="

# Define the number of pages you want to scrape, for instance 5 pages
num_pages = 5

# Create an empty DataFrame to store results
all_data = pd.DataFrame(columns=["Title", "Link"])

# Loop through pages
for page in range(1, num_pages + 1):
    # Construct full URL
    url = base_url + str(page)+"&regDate=20230923"

    # Get the webpage content
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors
    soup = BeautifulSoup(response.text, 'html.parser')
    # Extract news items
    news_items = soup.select('.box_etc li > div.cont_thumb > strong.tit_thumb')

    # Extract titles and links and append to the DataFrame
    for item in news_items:
        title = item.select_one('a.link_txt').text
        link = item.select_one('a.link_txt')['href']
        all_data = all_data.append({"Title": title, "Link": link}, ignore_index=True)



# Display the first few rows of the DataFrame for a quick check
print(all_data.head())


병렬 처리를 통한 속도 개선

In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Base URL
base_url = "https://news.daum.net/breakingnews/digital?page="

# Define the number of pages you want to scrape, for instance 5 pages
num_pages = 5

# Function to fetch and parse a single page
def fetch_page(page):
    url = base_url + str(page)
    with session.get(url) as response:
        response.raise_for_status()  # Raise an exception for HTTP errors
        soup = BeautifulSoup(response.text, 'html.parser')
        news_items = soup.select('li > div.cont_thumb > strong.tit_thumb')
        return [(item.select_one('a.link_txt').text, item.select_one('a.link_txt')['href']) for item in news_items]

# Create an empty list to store results
all_data_list = []

# Use a session for making requests
with requests.Session() as session:
    # Use ThreadPoolExecutor to parallelize the requests
    with ThreadPoolExecutor(max_workers=5) as executor:
        for data in executor.map(fetch_page, range(1, num_pages + 1)):
            all_data_list.extend(data)

# Convert the list to a DataFrame
all_data = pd.DataFrame(all_data_list, columns=["Title", "Link"])

# Display the first few rows of the DataFrame for a quick check
print(all_data)


                                       Title  \
0         "채널톡, 대한민국 대표 SaaS···글로벌서도 성과 낼 것"   
1                   '디플정'이 선정한 DX 톱3 기업 한자리에   
2                 MS·블리자드 합병 초읽기…英 "독점우려 해소"   
3             영하 100도의 밤 견디지 못하고…인도 달착륙선 잠들다   
4       에티버스 '클라우드 통합 보안 웨비나'에 약 100개 MSP 등록   
..                                       ...   
77  [애니멀리포트] 뇌 없어도 학습 능력 갖춘 해파리… 기억력의 기원 찾을까   
78                최종호 목사, 세상 영혼을 돌보는 '사회선교사'   
79           ‘파격 드레스’ 류호정이 띄운 지 2년째…타투법 근황은?   
80                        탈출 조종사 구조 요청 전화 공개   
81          흉기난동 공포에 지하철 아수라장…일본인 女관광객 결박 폭행   

                                      Link  
0   https://v.daum.net/v/20230923121206282  
1   https://v.daum.net/v/20230923120113200  
2   https://v.daum.net/v/20230923120008172  
3   https://v.daum.net/v/20230923115648138  
4   https://v.daum.net/v/20230923114918093  
..                                     ...  
77  https://v.daum.net/v/20230923000026386  
78  https://v.daum.net/v/20230923121800333  
79  https://v.daum

In [ ]:
a


### 뉴스 본문 클리핑 코드

병렬 처리를 통한 속도 개선

In [32]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

# Function to fetch and extract the main body of the article
def extract_article_content(url):
    with session.get(url) as response:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extracting the text from each <p> tag within the <section> tag
        article_paragraphs = soup.select("div.article_view section p")
        article_text = '\n'.join([para.text for para in article_paragraphs])
        return article_text if article_text else ""

# Extract the first 10 links (or however many you want)
links_to_scrape = all_data["Link"].tolist()

# Use a session for making requests
with requests.Session() as session:
    # Use ThreadPoolExecutor to parallelize the requests
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Since the order might be different due to parallel processing, we ensure the order is retained
        all_data["article"] = list(executor.map(extract_article_content, links_to_scrape))

all_data.head()


,Title,Link,article
0,"""채널톡, 대한민국 대표 SaaS···글로벌서도 성과 낼 것""",https://v.daum.net/v/20230923121206282,(지디넷코리아=방은주 기자) 2014년 설립된 채널코퍼레이션(대표 최시원) 글로벌 ...
1,'디플정'이 선정한 DX 톱3 기업 한자리에,https://v.daum.net/v/20230923120113200,(지디넷코리아=김미정 기자)디지털플랫폼정부위원회(DPG)가 선정한 디지털전환 톱3 ...
2,"MS·블리자드 합병 초읽기…英 ""독점우려 해소""",https://v.daum.net/v/20230923120008172,(지디넷코리아=김익현 미디어연구소장)690억 달러(약 92조원) 규모에 액티비전 블...
3,영하 100도의 밤 견디지 못하고…인도 달착륙선 잠들다,https://v.daum.net/v/20230923115648138,\n세계 최초로 달 남극에 착륙해 임무를 수행하던 인도의 달 착륙선과 탐사 로봇이 ...
4,에티버스 '클라우드 통합 보안 웨비나'에 약 100개 MSP 등록,https://v.daum.net/v/20230923114918093,통합 IT솔루션 전문 기업 에티버스는 지난 21일 '아크로니스'와 함께 국내 운영관...


In [ ]:
all_data

### api key 입력

In [ ]:
openai.api_key = ''

### 중요 기사 선별

In [ ]:
!pip install openai

In [68]:
import openai
import pandas as pd
import re

# OpenAI API 키 설정

# 엑셀 파일 불러오기
# file_path = "path_to_your_excel_file.xlsx"
# data = pd.read_excel(file_path)

data = all_data

def get_important_titles_for_chunk(chunk):
    titles = "\n".join(chunk)
    prompt_text = f"Given the following news titles, select the top 3 most important ones:\n{titles}\n\nTop 3 important news titles are:"
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt_text,
        max_tokens=200
    )
    important_titles = response.choices[0].text.strip().split("\n")
    return [re.sub(r"^\d+\.\s*", "", title) for title in important_titles]

# 10개씩 청크로 나누기
chunk_size = 10
chunks = [data['Title'][i:i+chunk_size].tolist() for i in range(0, len(data), chunk_size)]

all_important_titles = []
for chunk in chunks:
    all_important_titles.extend(get_important_titles_for_chunk(chunk))

# 최종적으로 원하는 숫자만큼 중요 기사 선택 (예: 상위 5개)
final_important_titles = all_important_titles[:5]

# 중요한 뉴스 제목에 해당하는 행들을 새로운 DataFrame으로 가져오기
important_news_df = data[data['Title'].isin(final_important_titles)]

print(important_news_df)


                                        Title  \
2                  MS·블리자드 합병 초읽기…英 "독점우려 해소"   
7   네이버, SME·크리에이터·스타트업 함께 성장하는 디지털 기술 생태계 조명   
9                   가상 아이돌 메이브의 팬 소통 기술은 'AI'   
10               유니티, 말 많던 ‘설치당 과금’ 소급 적용 않기로   
19          개발자 반발에 한발 물러선 유니티…'설치당 과금' 정책 개편   

                                      Link  \
2   https://v.daum.net/v/20230923120008172   
7   https://v.daum.net/v/20230923110138617   
9   https://v.daum.net/v/20230923110003586   
10  https://v.daum.net/v/20230923105207518   
19  https://v.daum.net/v/20230923100300012   

                                              article  
2   (지디넷코리아=김익현 미디어연구소장)690억 달러(약 92조원) 규모에 액티비전 블...  
7   \n‘디지털 생태계 리포트 2023’에 따르면, 네이버의 클라우드, AI, 빅데이터...  
9   메타버스엔터테인먼트-업스테이지, 페르소나 AI 공동 개발\n\n넷마블 자회사 메타버...  
10  콘텐츠 제작 엔진 개발사 유니티가 전 세계 게임 개발자들의 반발을 샀던 설치당 과금...  
19  \n(서울=연합뉴스) 김주환 기자 = 게임 설치 횟수에 따라 개발자들에게 요금을 청...  


In [ ]:
important_news_df

In [ ]:
# 엑셀 파일로 저장
# Save DataFrame to Excel
file_name = "daum_news_multiple_pages.xlsx"
all_data.to_excel(file_name, index=False)

# If using Colab, download the Excel file
from google.colab import files
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### gpt를 통한 요약

In [ ]:
!pip install openai

In [78]:
import openai

def summarize_news(news_body):

    # Define the conversation with the user's instruction to summarize the news body
    conversation = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"주어진 기사를 짧게 요약해줘 한글로: {news_body}"}
    ]

    # Set the maximum number of tokens for the summary (adjust as needed)
    max_tokens = 100

    # Make a request to the ChatGPT API
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=conversation,
    )
    # Extract and print the summary
    summary = response["choices"][0]["message"]["content"]
    return summary

def main(news_body):
    # Define the scraped news body you want to summarize
    # Summarize the news
    summary = summarize_news(news_body)
    # print("original:")
    # print(news_body)
    print("Summary:")
    print(summary)
    return summary
# 뉴스 요약
summary_list = important_news_df['article'].apply(main)

important_news_df['summary']=summary_list

Summary:
마이크로소프트는 액티비전 블리자드 인수에 필요한 수정안을 제출하였으며, 영국 경쟁시장국(CMA)은 해당 수정안을 검토할 예정이라고 밝혔다. 마이크로소프트는 액티비전 블리자드의 클라우드 게임 스트리밍 권한을 15년 동안 프랑스 게임회사 유비소프트에 넘기기로 결정하였다. 이에 따라 영국 CMA는 클라우드 게임 시장의 경쟁이 계속 유지될 수 있다는 평가를 내렸다. 영국 CMA는 10월 6일까지 마이크로소프트의 제안을 검토할 예정이다. 마이크로소프트는 액티비전 블리자드와의 합병 계획을 전 세계 주요 시장에서 이미 승인받았으며, 영국이 마지막으로 합병에 반대하는 국가였다.
Summary:
네이버의 클라우드, AI, 빅데이터 등 기술을 활용하는 파트너들의 범위와 지원 규모가 확대되고 있다. 네이버의 전략 투자 조직인 D2SF가 투자한 스타트업은 4조원 가치이며, AI 프로젝트에 참여한 스타트업을 위한 지원 규모는 20억원 이상이다. 또한, 외부 스타트업의 입점 비중은 65%에 달하며, 학교에서는 1.7만개의 웨일스페이스를 도입했다. 네이버의 온라인 교육프로그램인 부스트코스의 누적 수강자는 45만명이다. SME와 크리에이터도 네이버의 기술을 활용해 성장하고 있으며, 네이버 스마트스토어에서 연 1억원 이상의 매출을 올린 판매자는 4.5만명이다. 또한, 네이버페이의 빠른 정산을 통해 SME를 지원하고 있는 지원금액은 1060억원 이상이다. 네이버웹툰과 제페토 크리에이터 생태계도 성장하고 있으며, 네이버웹툰의 수익 모델인 PPS 프로그램의 규모는 2조255억원이다. 이러한 결과는 네이버의 다양한 리포트와 생태계 참여자들의 연구로부터 나왔다.
Summary:
넷마블 자회사 메타버스엔터테인먼트와 업스테이지가 '페르소나 AI'를 공동 개발할 예정이다. 페르소나 AI는 아티스트의 캐릭터와 정체성을 학습하고 팬과 소통하는 기술이다. 이를 통해 가상 걸그룹 '메이브'와 팬들이 대화를 나눌 수 있게 될 것이다. 또한, 액토즈소프트는 셩취게임즈와 중국 지역에서 '미르의 전설2

<ipython-input-78-f8d8b6201167>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  important_news_df['summary']=summary_list


In [80]:
important_news_df

,Title,Link,article,summary
2,"MS·블리자드 합병 초읽기…英 ""독점우려 해소""",https://v.daum.net/v/20230923120008172,(지디넷코리아=김익현 미디어연구소장)690억 달러(약 92조원) 규모에 액티비전 블...,"마이크로소프트는 액티비전 블리자드 인수에 필요한 수정안을 제출하였으며, 영국 경쟁시..."
7,"네이버, SME·크리에이터·스타트업 함께 성장하는 디지털 기술 생태계 조명",https://v.daum.net/v/20230923110138617,"\n‘디지털 생태계 리포트 2023’에 따르면, 네이버의 클라우드, AI, 빅데이터...","네이버의 클라우드, AI, 빅데이터 등 기술을 활용하는 파트너들의 범위와 지원 규모..."
9,가상 아이돌 메이브의 팬 소통 기술은 'AI',https://v.daum.net/v/20230923110003586,"메타버스엔터테인먼트-업스테이지, 페르소나 AI 공동 개발\n\n넷마블 자회사 메타버...",넷마블 자회사 메타버스엔터테인먼트와 업스테이지가 '페르소나 AI'를 공동 개발할 예...
10,"유니티, 말 많던 ‘설치당 과금’ 소급 적용 않기로",https://v.daum.net/v/20230923105207518,콘텐츠 제작 엔진 개발사 유니티가 전 세계 게임 개발자들의 반발을 샀던 설치당 과금...,게임 개발 엔진 개발사인 유니티가 설치당 과금 요금제에 대한 반발로 거스른 결정을 ...
19,개발자 반발에 한발 물러선 유니티…'설치당 과금' 정책 개편,https://v.daum.net/v/20230923100300012,\n(서울=연합뉴스) 김주환 기자 = 게임 설치 횟수에 따라 개발자들에게 요금을 청...,게임 엔진 '유니티'가 개발자들을 위한 새로운 가격 정책 '런타임 수수료'를 발표했...


## keyword 추출

In [81]:
import openai
import pandas as pd


# 중요한 기사들의 DataFrame
important_news_df = important_news_df  # 이미 정의된 DataFrame

def extract_keywords_from_content(content):
    # 본문의 시작 부분과 끝 부분을 결합
    truncated_content = content[:600] + "\n...\n" + content[-600:]

    prompt_text = f"주어진 뉴스 기사의 본문을 보고 핵심 키워드를 `핵심 키워드 : 축구, 이강인, 멘체스터 유나이티드' 와 같은 형태로 추출해줘. ###{truncated_content}###"
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt_text,
        max_tokens=200  # 조절 가능
    )
    keywords = response.choices[0].text.strip().split(", ")
    keywords[0]=keywords[0].replace("핵심 키워드","")
    keywords[0]=keywords[0].replace(":","").strip()
    return keywords

# 각 본문에 대해 주요 키워드를 추출
keywords_list = important_news_df['article'].apply(extract_keywords_from_content)

# 새로운 DataFrame 생성
keywords_df = important_news_df.copy()

keywords_df['keywords']=keywords_list
print(keywords_df)


                                        Title  \
2                  MS·블리자드 합병 초읽기…英 "독점우려 해소"   
7   네이버, SME·크리에이터·스타트업 함께 성장하는 디지털 기술 생태계 조명   
9                   가상 아이돌 메이브의 팬 소통 기술은 'AI'   
10               유니티, 말 많던 ‘설치당 과금’ 소급 적용 않기로   
19          개발자 반발에 한발 물러선 유니티…'설치당 과금' 정책 개편   

                                      Link  \
2   https://v.daum.net/v/20230923120008172   
7   https://v.daum.net/v/20230923110138617   
9   https://v.daum.net/v/20230923110003586   
10  https://v.daum.net/v/20230923105207518   
19  https://v.daum.net/v/20230923100300012   

                                              article  \
2   (지디넷코리아=김익현 미디어연구소장)690억 달러(약 92조원) 규모에 액티비전 블...   
7   \n‘디지털 생태계 리포트 2023’에 따르면, 네이버의 클라우드, AI, 빅데이터...   
9   메타버스엔터테인먼트-업스테이지, 페르소나 AI 공동 개발\n\n넷마블 자회사 메타버...   
10  콘텐츠 제작 엔진 개발사 유니티가 전 세계 게임 개발자들의 반발을 샀던 설치당 과금...   
19  \n(서울=연합뉴스) 김주환 기자 = 게임 설치 횟수에 따라 개발자들에게 요금을 청...   

                                              summary  \
2   마이크로소프트는 액티비전 블리자드 인수에 필

## 메일 발송

In [82]:
import openai
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart


# 데이터프레임의 내용을 HTML 형식으로 변환
email_content = keywords_df.to_html()

# DataFrame을 원하는 형태로 리스트로 변환
html_content = ""
for index, row in keywords_df.iterrows():
    title = row["Title"]
    link = row["Link"]
    summary = row["summary"]
    keyword = row["keywords"]

    # 하이퍼링크 추가
    title_with_link = f'<a href="{link}">{title}</a>'

    # 형식에 맞게 포맷팅
    formatted_item = f"<h2>제목 : {title_with_link}</h2><br>요약<br>{summary}<br>키워드: {keyword[:5]}<br>"

    html_content += formatted_item

# 이메일 설정
sender_email = "myemail"
receiver_email = "recevier email"
password = ""
subject = "DataFrame Contents"

message = MIMEMultipart("alternative")
message["Subject"] = subject
message["From"] = sender_email
message["To"] = receiver_email
part = MIMEText(html_content, "html")
message.attach(part)

# 이메일 보내기 naver email 이라서, 2차 비밀번호인 JPSS63MKXHT3 사용
with smtplib.SMTP_SSL("smtp.naver.com", 465) as server:
    server.login(sender_email, 'JPSS63MKXHT3')
    server.sendmail(sender_email, receiver_email, message.as_string())
